In [1]:
import numpy as np # 선형 대수 연산용
import pandas as pd # 데이터 처리, CSV 파일 입출력용 (예: pd.read_csv)
# 현재 디렉토리(/kaggle/working/)에는 최대 20GB까지 파일을 저장할 수 있으며,
# "Save & Run All" 기능을 사용하여 버전을 만들 때 출력 결과로 보존됩니다
# 또한 /kaggle/temp/ 디렉토리에는 임시 파일을 저장할 수 있으나,
# 이 파일들은 현재 세션이 종료되면 저장되지 않습니다

In [2]:
import os  # 운영 체제와 상호작용하기 위한 모듈 (예: 파일 경로 설정, 디렉터리 생성 등)

import tensorflow as tf  # 구글에서 만든 딥러닝 라이브러리 TensorFlow를 불러옴
from tensorflow import keras  # TensorFlow 안에 포함된 고수준 딥러닝 API인 Keras를 불러옴

from tensorflow.keras.optimizers import Adam  # 모델 학습 시 사용하는 최적화 알고리즘 중 하나인 Adam 불러옴

from sklearn.model_selection import train_test_split  # 데이터를 학습용과 테스트용으로 나누기 위한 함수

from tensorflow.keras.metrics import categorical_crossentropy  # 다중 분류 문제에서 사용되는 손실 함수 (정답과 예측 차이 계산)

from tensorflow.keras.models import Model, load_model, Sequential  # Keras의 모델 관련 기능 불러옴 (Model: 기본 모델, load_model: 저장된 모델 불러오기, Sequential: 레이어를 순서대로 쌓는 모델)

from sklearn.metrics import confusion_matrix, classification_report  # 모델의 성능 평가에 사용되는 도구 (혼동 행렬, 정확도 등)

from tensorflow.keras.preprocessing.image import ImageDataGenerator  # 이미지 데이터를 증강(늘리기)하기 위한 도구 (회전, 확대, 이동 등으로 이미지 다양화)

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization  
# Conv2D: 이미지 특징을 추출하는 필터 레이어  
# MaxPooling2D: 이미지 크기를 줄이면서 중요한 특징만 남기는 레이어  
# Flatten: 2D 이미지를 1D로 펴주는 레이어 (Dense 레이어 입력을 위해 필요)  
# Dense: 완전 연결된 레이어 (신경망의 기본 구조)  
# Activation: 활성화 함수 적용 (예: ReLU, Softmax 등)  
# Dropout: 과적합을 막기 위해 일부 뉴런을 학습 중 무시하는 기법  
# BatchNormalization: 학습 속도 향상 및 안정화를 위한 정규화 레이어


In [3]:
import os  # 운영체제 기능을 사용하기 위한 모듈을 불러옴 (예: 폴더 경로 탐색 등)

# 데이터셋이 저장된 폴더 경로를 지정함
fpath = "./archive/train"
dataset_path = fpath  # 위에서 지정한 경로를 dataset_path 변수에 저장

# 폴더 안에 있는 하위 디렉토리 목록을 가져옴 (각 디렉토리는 한 개의 클래스에 해당됨)
class_folders = [f.name for f in os.scandir(dataset_path) if f.is_dir()]  
# 예: 'apple', 'tomato'라는 폴더가 있다면 ['apple', 'tomato'] 반환

# 클래스(분류할 대상)의 개수를 계산함
num_classes = len(class_folders)  # 위에서 찾은 폴더 개수만큼 클래스 수가 정해짐

# 클래스의 개수를 출력함
print("Number of classes:", num_classes)  # 예: Number of classes: 2

# 클래스 이름들을 리스트로 저장 (예: ['apple', 'tomato'])
target_names = class_folders  # 예측 결과를 사람이 이해할 수 있도록 이름으로 저장

# 클래스 이름 리스트를 출력함
print("Target Names:", target_names)  # 예: Target Names: ['apple', 'tomato']


Number of classes: 2
Target Names: ['apples', 'tomatoes']


In [4]:
img_size = 224
batch_size =50
fpath = "./archive/train"
ffpath="./archive/test"

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # 이미지 데이터 증강을 위한 도구를 불러옴 (회전, 확대, 정규화 등)

# 'fpath'는 학습용 데이터셋 폴더 경로이고, 'batch_size'는 한 번에 불러올 이미지 수라고 가정

# 학습 및 검증 데이터를 전처리하고, 이미지 증강을 설정함
datagen = ImageDataGenerator(
    rescale=1/255.,              # 픽셀 값을 0~255 → 0~1로 정규화 (모델 학습에 적합하게 만듦)
    zoom_range=0.2,              # 이미지를 랜덤하게 최대 20%까지 확대
    rotation_range=30,          # 이미지를 -30도~+30도 사이로 회전시킴
    validation_split=0.2,       # 전체 데이터 중 20%는 검증용(validation)으로 사용
    horizontal_flip=True        # 이미지를 좌우로 뒤집는 데이터 증강 적용
)

# 학습용 이미지 생성기 설정
train_gen = datagen.flow_from_directory(
    fpath,                      # 이미지가 저장된 최상위 폴더 경로
    target_size=(224, 224),     # 모든 이미지를 224x224 크기로 맞춤 (모델 입력 크기)
    subset="training",          # 전체 중 80%를 학습용으로 사용
    class_mode='categorical',   # 다중 클래스 분류이므로 one-hot 인코딩된 label 사용
    batch_size=batch_size       # 한 번에 불러올 이미지 개수 (성능과 속도에 영향을 줌)
)

# 검증용 이미지 생성기 설정
valid_gen = datagen.flow_from_directory(
    fpath,                      # 같은 폴더에서 검증용 이미지도 불러옴
    target_size=(224, 224),     # 이미지 크기 통일
    batch_size=batch_size,      # 배치 크기 동일하게 설정
    subset='validation',        # 전체 중 20%를 검증용으로 사용
    class_mode='categorical'    # 다중 분류를 위한 one-hot 인코딩
)

# 테스트용 데이터 생성기 설정
test_gen = ImageDataGenerator(rescale=1/255.).flow_from_directory(
    ffpath,                     # 테스트 이미지가 저장된 폴더 경로 (오타 주의: fpath와 다름)
    target_size=(224, 224),     # 테스트 이미지도 크기를 동일하게 맞춤
    batch_size=50,              # 한 번에 50장의 이미지를 불러옴
    class_mode='categorical',   # 다중 클래스 분류용 설정
    shuffle=False               # 테스트에서는 결과를 정확히 비교해야 하므로 셔플 비활성화
)


Found 236 images belonging to 2 classes.
Found 58 images belonging to 2 classes.
Found 97 images belonging to 2 classes.


In [12]:
import tensorflow as tf  # 텐서플로(TensorFlow)는 딥러닝을 구현할 수 있는 구글의 오픈소스 프레임워크
from tensorflow.keras.models import Sequential  # 레이어를 순차적으로 쌓을 수 있는 간단한 모델 형식
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization  # 완전 연결층(Dense), 과적합 방지용 드롭아웃, 학습 안정화를 위한 배치 정규화 레이어
from tensorflow.keras.callbacks import ModelCheckpoint  # 모델을 저장할 수 있는 콜백 함수 불러오기

# train_gen과 valid_gen은 이미지 데이터 생성기라고 가정 (학습용과 검증용 이미지)

img_size = (224, 224)  # 이미지의 크기를 224x224로 설정 (입력으로 통일된 크기 사용)
channels = 3  # 컬러 이미지이므로 R, G, B 채널로 총 3개
img_shape = (img_size[0], img_size[1], channels)  # 전체 입력 이미지의 형태 (224, 224, 3)

class_count = len(list(train_gen.class_indices.keys()))  # 분류해야 할 클래스(카테고리) 수를 계산함

# 사전 학습된 모델(EfficientNetB3)을 불러옴
base_model = tf.keras.applications.EfficientNetB3(
    include_top=False,  # 기본 모델의 마지막 분류 레이어는 제거
    weights="imagenet",  # ImageNet 데이터로 학습된 가중치를 사용
    input_shape=img_shape,  # 입력 이미지의 크기를 지정
    pooling='max'  # 마지막 특성 맵을 전역 최대값으로 압축
)

# 사전학습된 레이어들의 가중치를 고정시켜 학습되지 않도록 설정
for layer in base_model.layers:
    layer.trainable = False  # 가중치 업데이트를 하지 않도록 설정 (특징 추출용으로만 사용)

# 전체 모델 구성: 사전학습 모델 위에 새 레이어들을 쌓아 최종 분류 모델 생성
model = Sequential([
    base_model,                         # 사전학습된 베이스 모델
    BatchNormalization(),              # 학습 안정화를 위한 배치 정규화
    Dense(256, activation='relu'),     # 은닉층 256개 뉴런, 활성화 함수는 ReLU
    Dropout(0.3),                      # 학습 중 30% 뉴런을 비활성화하여 과적합 방지
    Dense(64, activation='relu'),      # 또 다른 은닉층 64개 뉴런
    Dropout(0.3),                      # 과적합 방지를 위한 드롭아웃
    Dense(class_count, activation='softmax')  # 출력층: 클래스 수만큼 뉴런, 확률값을 출력하기 위해 softmax 사용
])

# 모델 학습을 위한 최적화 알고리즘(Adam) 설정
optimizer = tf.keras.optimizers.Adam()

# 모델 컴파일: 손실 함수, 최적화 알고리즘, 평가 지표 설정
model.compile(
    optimizer=optimizer,                     # 위에서 정의한 Adam 최적화기 사용
    loss='categorical_crossentropy',         # 다중 분류이므로 범주형 크로스 엔트로피 사용
    metrics=['accuracy']                     # 모델 성능을 정확도로 평가
)

# 모델을 저장할 파일 경로 지정
filepath = 'best_model.h5'  # 가장 성능이 좋은 모델을 이 파일명으로 저장함

# 검증 정확도를 기준으로 가장 좋은 모델만 저장하는 콜백 정의
checkpoint = ModelCheckpoint(
    filepath,                # 저장 파일 경로
    monitor='val_accuracy', # 검증 데이터의 정확도를 기준으로 판단
    verbose=1,              # 저장 여부를 콘솔에 출력함
    save_best_only=True,    # 정확도가 가장 높을 때만 저장
    mode='max'              # 최대값이 가장 좋은 성능으로 간주
)

# 모델을 학습하고, 콜백을 적용하여 최적의 모델을 자동으로 저장함
history = model.fit(
    x=train_gen,            # 학습 데이터 생성기
    epochs=2000,              # 전체 데이터셋을 20번 반복 학습
    verbose=1,              # 학습 과정을 자세히 출력
    validation_data=valid_gen,  # 검증용 데이터 생성기
    callbacks=[checkpoint]      # 체크포인트 콜백을 적용하여 모델 저장
)


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average



model= load_model('best_model.h5')

train_score = model.evaluate(train_gen, steps= len(train_gen), verbose= 1)
valid_score = model.evaluate(valid_gen, steps= len(valid_gen), verbose= 1)
test_score = model.evaluate(test_gen, steps= len(test_gen), verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
model = load_model('best_model.h5')
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds, axis=1)
print(y_pred)

In [ ]:
import itertools
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes, normalize= False, title= 'Confusion Matrix', cmap= plt.cm.Blues):
    plt.figure(figsize= (10, 10))
    plt.imshow(cm, interpolation= 'nearest', cmap= cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation= 45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis= 1)[:, np.newaxis]
        print('Normalized Confusion Matrix')
    else:
        print('Confusion Matrix, Without Normalization')
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= target_names, title = 'Confusion Matrix')
# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= target_names))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

# Assuming train_gen and valid_gen are your image data generators

img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys()))

# Create pre-trained model
base_model = tf.keras.applications.ResNet50(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')
for layer in base_model.layers:
    layer.trainable = False
model = Sequential([
    base_model,
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.3),
     Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(class_count, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define filepath to save the best model
filepath = 'best_model_resnet.h5'

# Create ModelCheckpoint callback to save the best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# Train the model with the added callback
history = model.fit(
    x=train_gen,
    epochs=20,
    verbose=1,
    validation_data=valid_gen,
    callbacks=[checkpoint]
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average



model= load_model('best_model_resnet.h5')


train_score = model.evaluate(train_gen, steps= len(train_gen), verbose= 1)
valid_score = model.evaluate(valid_gen, steps= len(valid_gen), verbose= 1)
test_score = model.evaluate(test_gen, steps= len(test_gen), verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
model = load_model('best_model_resnet.h5')
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds, axis=1)
print(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= target_names, title = 'Confusion Matrix')
# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= target_names))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

# Assuming train_gen and valid_gen are your image data generators

img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys()))

# Create pre-trained model
base_model = tf.keras.applications.VGG16(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')
for layer in base_model.layers:
    layer.trainable = False
model = Sequential([
    base_model,
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.3),
     Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(class_count, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define filepath to save the best model
filepath = 'best_model_vgg.h5'

# Create ModelCheckpoint callback to save the best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# Train the model with the added callback
history = model.fit(
    x=train_gen,
    epochs=20,
    verbose=1,
    validation_data=valid_gen,
    callbacks=[checkpoint]
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average



model= load_model('best_model_vgg.h5')


train_score = model.evaluate(train_gen, steps= len(train_gen), verbose= 1)
valid_score = model.evaluate(valid_gen, steps= len(valid_gen), verbose= 1)
test_score = model.evaluate(test_gen, steps= len(test_gen), verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
model = load_model('best_model_vgg.h5')
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds, axis=1)
print(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= target_names, title = 'Confusion Matrix')
# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= target_names))